# **ETL del dataset:** *users_items*

#### **Importación de librerías** ####
---

*Para este cuaderno usaremos las siguientes librerías: **pandas**, **gzip**, **ast**, **numpy**y **pyarrow**.*

In [1]:
import pandas as pd
import gzip
import ast
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq

#### **Extracción de los datos** ####
---

*Procedemos a abrir nuestro dataset y almacenarlo en un dataframe para visualizarlo y manejarlo en pandas.*

1. *Encontramos un error en el formato del dataset que no permite abrirlo con la **forma tradicional**.*
2. *Al extraer los datos **todas las filas tienen error** de importación, por lo tanto lo haremos de otra forma.*
3. *Creamos una iteración que lea cada linea y con la función **literal_eval** convertir el tipo de dato a uno reconocible por Python*

In [2]:
ruta = 'datasets/users_items.json.gz'

In [3]:
listData = [] 

for line in gzip.open(ruta):
    listData.append(ast.literal_eval(line.decode("utf-8")))

In [4]:
dfUsersItems = pd.DataFrame(listData)
dfUsersItems.head(3)

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."


*Procedemos a normalizar los datos de la columna items:*

1. *La columna items contiene datos que se encuentran **anidados**, así que debemos normalizarlos.*
2. *Usamos la función **json_normalize** para explotar estos datos y transformarlos en columnas.*
3. *Usamos el parametro **meta** para que la función nos conserve las columnas vistas anteriormente.*

In [5]:
dfUsersItems= pd.json_normalize(listData, record_path=['items'], meta=['steam_id','items_count','user_id', 'user_url'] )
dfUsersItems.head(5)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


### **Análisis Exploratorio Inicial** ###
---

*Vamos a realizar una **exploración de los datos** (EDA) antes de hacer transformación de los mismos, esto con el fin de **conocer las columnas** y determinar que campos son valiosos para nuestro análisis y así tener claro que pasos realizaremos en el ETL.*

**Nota:**: *Posterior al ETL realizaremos también un [EDA](04_EDA.ipynb)
 enfocado en los datos y una descripción estadística.*

#### **Revisión de la extracción de los datos** ####

*Revisamos que los datos se cargaron adecuadamente en nuestro dataframe:*

1. *Usamos **head** para previsualizar los primeros registros del Dataframe.*
2. *Podemos observar que se cargaron adecuadamente los datos y no se ven por ahora anomalías en estos.*

In [6]:
dfUsersItems.head(5)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...


*Revisemos si este patrón se repite en todos los datos:*

1. *Usamos **tail** para previsualizar los últimos registros del Dataframe.*
2. *No se observan anomalías por ahora.*

In [7]:
dfUsersItems.tail(3)

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
5153206,388490,One Way To Die: Steam Edition,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153208,519140,Minds Eyes,3,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


*Revisamos la forma del dataframe:*

1. *Usamos **shape** obtener el número de filas y columnas del dataframe.*
2. *Nuestro dataframe tiene **5153209 filas y 8 columnas**.*

In [8]:
dfUsersItems.shape

(5153209, 8)

#### **Exploración de las columnas** ####

*Vamos a revisar qué columnas existen en el dataframe y de qué tipo son:*

1. *Usamos el método **dtypes** para listar las columnas y sus tipos.*
2. *Observamos que dos de las columnas contienen datos numéricos identificados como **int64**.*
3. *Las demás columnas son de tipo **object**.*

In [9]:
dfUsersItems.dtypes

item_id             object
item_name           object
playtime_forever     int64
playtime_2weeks      int64
steam_id            object
items_count         object
user_id             object
user_url            object
dtype: object

*Revisemos qué tipos de datos almacena cada columna a detalle:*

1. *Hacemos una **iteración** que nos permite recorrer los valores de cada columna para encontrar sus tipos.*
2. *Observamos que las columnas **playtime** son de tipo int, al igual que la columna **items_count**.*
3. *La columna **item_id** es de tipo string, pero podríamos convertirla a int.*

In [10]:
tipoDatos = {"Columna":[], "Tipos":[]}
for c in dfUsersItems.columns:
    tipoDatos["Columna"].append(c)
    tipoDatos["Tipos"].append(dfUsersItems[c].apply(type).unique())
    
dfTypes = pd.DataFrame(tipoDatos)
dfTypes

,Columna,Tipos
0,item_id,[<class 'str'>]
1,item_name,[<class 'str'>]
2,playtime_forever,[<class 'int'>]
3,playtime_2weeks,[<class 'int'>]
4,steam_id,[<class 'str'>]
5,items_count,[<class 'int'>]
6,user_id,[<class 'str'>]
7,user_url,[<class 'str'>]


*Revisemos las columnas consideradas como numéricas:*

1. *Usamos **describe()** para obtener un resumen de estas columnas.*
2. *Por ahora no dicen mucho sobre nuestros datos ya que podríans ser columnas no muy útiles para nuestro análisis.*

In [11]:
dfUsersItems.describe()

,playtime_forever,playtime_2weeks
count,5.153209e+06,5.153209e+06
mean,9.914951e+02,9.104707e+00
std,5.418204e+03,1.403926e+02
min,0.000000e+00,0.000000e+00
25%,0.000000e+00,0.000000e+00
50%,3.400000e+01,0.000000e+00
75%,3.550000e+02,0.000000e+00
max,6.427730e+05,1.996700e+04


*Revisemos los valores alojados en cada columna para saber cuantos son validos, es decir no nulos:*

1. *Usamos **info()** para obtener un conteo de los valores válidos por columnas*
2. *Sin embargo este no nos trae un valor aproximado, posiblemente por el número de filas.*

In [12]:
dfUsersItems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 8 columns):
 #   Column            Dtype 
---  ------            ----- 
 0   item_id           object
 1   item_name         object
 2   playtime_forever  int64 
 3   playtime_2weeks   int64 
 4   steam_id          object
 5   items_count       object
 6   user_id           object
 7   user_url          object
dtypes: int64(2), object(6)
memory usage: 314.5+ MB


*Revisemos las columnas en porcentajes de valores nulos:*

1. *Usamos **isnull()** y **sum()** para obtener una suma de los valores nulos por columna.*
2. *Observamos que entre el **70% y 80%** de los regitros de cada columna son vacíos.*

In [13]:
nulosPorCol = dfUsersItems.isnull().sum()
porcentajePorCol = (nulosPorCol/dfUsersItems.shape[0])*100
porcentajePorCol

item_id             0.0
item_name           0.0
playtime_forever    0.0
playtime_2weeks     0.0
steam_id            0.0
items_count         0.0
user_id             0.0
user_url            0.0
dtype: float64

*Descripción de las variables del dataframe:*

1. *Creamos una tabla para **describir y comparar las columnas**.*

| Variable          | Definición                                             | Tipo de Dato | Tipo de Variable    |
|:------------------|:-------------------------------------------------------|:------------:|:----------------    |
| Item Id           | Identificador único de la aplicación(videojuego)       | Integer      | Numérica(discreta)  |
| Item Name         | Es el nombre de la aplicación                          | String       | Categórica(nominal) |
| Playtime Forever	| Cantidad de tiempo jugando la aplicacióne specífica    | Integer      | Numérica(discreta)  |
| Playtime 2 weeks	| Cantidad de tiempo jugado en las últimas dos semanas   | Integer      | Numérica(discreta)  |
| Steam id          | Identificador único de usuario asignado por Steam      | Integer      | Numérica(discreta)  |
| User id           | Identificador único de usuario asignado por el usuario | String       | Catégorica(nominal) |
| User url          | Enlace al perfil del usuario en Steam                  | String       | Referencia a la web |


2. *Revisamos las especificaciones de las funciones que requiere nuestro **MVP** para saber que columnas serán importantes para las funcionalidades de la API.*
3. *Observamos que existen columnas como **User Url, playtime*** que no serán apropiadas para los objetivos de este análisis.*

**Nota:** *En algunos visualizadores de Markdown no es posible apreciar la tabla, sugiero revisarla VSC.*

### **Transformación de los datos** ###
---

*A partir de este momento realizaremos una limpieza de los datos teniendo en cuenta nuestro análisis previo.*

* **Nota:** *Posterior al ETL realizaremos también un [EDA](04_EDA.ipynb)
 enfocado en los datos y una descripción estadística.*

#### Eliminación de datos nulos ####

*Procedemos a eliminar las filas con registros nulos:*

1. *Usamos **dropna()** para eliminar filas con registros nulos*.
2. *El número de filas no cambió, ya que no se encontraron registros nulos*.

In [14]:
dfUsersItems.shape

(5153209, 8)

In [15]:
dfUsersItems = dfUsersItems.dropna()
dfUsersItems.shape

(5153209, 8)

#### Eliminación de duplicados de datos ####

*Procedemos a revisar si nos quedan filas cuyos valores sean duplicados:*

1. *Usamos **duplicated()** para encontrar filas con registros nulos usando la columna "id"*.
2. *Observamos que nos muestra registros duplicados por **steam_id** sin embargo es entendible porque un jugador puede jugar más de un juego.*
3. *Tomamos la decisión de **no eliminar** registros.*

In [16]:
duplicados = dfUsersItems[dfUsersItems.duplicated()]
duplicados

,item_id,item_name,playtime_forever,playtime_2weeks,steam_id,items_count,user_id,user_url
164294,20,Team Fortress Classic,5,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164295,50,Half-Life: Opposing Force,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164296,70,Half-Life,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164297,130,Half-Life: Blue Shift,0,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
164298,220,Half-Life 2,198,0,76561198084006094,109,Nikiad,http://steamcommunity.com/id/Nikiad
...,...,...,...,...,...,...,...,...
4898223,213670,South Park™: The Stick of Truth™,725,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898224,221910,The Stanley Parable,53,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898225,261030,The Walking Dead: Season Two,253,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...
4898226,273110,Counter-Strike Nexon: Zombies,0,0,76561198080057659,39,76561198080057659,http://steamcommunity.com/profiles/76561198080...


#### Tranformación por columnas ####

*A continuación vamos a hacer limpieza por columnas, pero antes haremos algunos cambios:*

1. **Reorganizamos las columnas** *, con el fin de mejorar la lectura y comparación de los datos y columnas*.
2. **Cambiamos los nombres** *de las columnas.*

In [17]:
dfUsersItems.columns

Index(['item_id', 'item_name', 'playtime_forever', 'playtime_2weeks',
       'steam_id', 'items_count', 'user_id', 'user_url'],
      dtype='object')

In [18]:
dfUsersItems = dfUsersItems[["steam_id", "user_id", "item_id","item_name", "items_count", "playtime_forever", "playtime_2weeks", "user_url"]]

In [19]:
dfUsersItems.columns = ['IdUserSteam', 'IdUser', "IdApp", "NameApp", "Count", "Playtime", "PlaytimeLast2W", "UserUrl"]

**Columnas IdUserSteam y IdApp**

1. *Hacemos un conteo de valores para observar cuantas veces se repiten y la cantidad total de usuarios es de **70.912**.*
2. *Cambiamos el tipo de dato a **int**.*
3. *Reorganizamos los datos por **id**.*
3. *Obtenemos información valiosa al ordenar los valores de esta forma, observamos que los registros que contengan playtime = 0 son juegos comprados por el usuario que no han sido jugados.*

In [20]:
dfUsersItems.IdUserSteam.value_counts()

IdUserSteam
76561198102767019    7762
76561198000537256    6700
76561198001518866    6410
76561198118965684    6132
76561198048151962    5027
                     ... 
76561198092083433       1
76561198062649642       1
76561198092078142       1
76561198062660594       1
76561198063378419       1
Name: count, Length: 70912, dtype: int64

In [21]:
dfUsersItems["IdUserSteam"] = dfUsersItems["IdUserSteam"].astype(np.int64)
dfUsersItems["IdApp"] = dfUsersItems["IdApp"].astype(np.int64)

In [22]:
dfUsersItems.sort_values(by="IdUserSteam", inplace=True)
dfUsersItems.reset_index(inplace=True, drop=True)
dfUsersItems.head(3)

,IdUserSteam,IdUser,IdApp,NameApp,Count,Playtime,PlaytimeLast2W,UserUrl
0,76561197960269200,ONIONZZZ,306660,Ultimate General: Gettysburg,384,0,0,http://steamcommunity.com/id/ONIONZZZ
1,76561197960269200,ONIONZZZ,12150,Max Payne 2: The Fall of Max Payne,384,0,0,http://steamcommunity.com/id/ONIONZZZ
2,76561197960269200,ONIONZZZ,4700,Medieval II: Total War,384,4,0,http://steamcommunity.com/id/ONIONZZZ


**Columnas IdUser, UserUrl y Count**

1. *Observamos que estas columnas pueden mostrarse en otro dataframe con la información del usuario, así que dividiremos el dataset*.
2. *La columna count cuenta el número de apps que ha comprado el usuario, por lo tanto este valor tiende a repetirse cada vez que se repite el usuario.*

In [23]:
dfUsers = dfUsersItems[["IdUserSteam", "IdUser", "Count", "UserUrl"]].copy()

In [24]:
dfUsers.head(3)

,IdUserSteam,IdUser,Count,UserUrl
0,76561197960269200,ONIONZZZ,384,http://steamcommunity.com/id/ONIONZZZ
1,76561197960269200,ONIONZZZ,384,http://steamcommunity.com/id/ONIONZZZ
2,76561197960269200,ONIONZZZ,384,http://steamcommunity.com/id/ONIONZZZ


**Dataframe dfUsers**

1. *En este dataset procedemos a eliminar los i**d de usuario repetidos**.*
2. *La columna count se llamará **PurchasedItems**.*
3. *Cambiamos el tipo de la columna PruchasedItems a **int**.*

In [25]:
dfUsers.drop_duplicates(subset=["IdUserSteam"],inplace=True)
dfUsers.sort_values(by="IdUserSteam", inplace=True)
dfUsers.reset_index(inplace=True, drop=True)
dfUsers.head(3)

,IdUserSteam,IdUser,Count,UserUrl
0,76561197960269200,ONIONZZZ,384,http://steamcommunity.com/id/ONIONZZZ
1,76561197960270504,exaqt,209,http://steamcommunity.com/id/exaqt
2,76561197960270742,dvs,36,http://steamcommunity.com/id/dvs


In [26]:
dfUsers.rename(columns={"Count":"PurchasedItems"}, inplace=True)
dfUsers["PurchasedItems"] = dfUsers["PurchasedItems"].astype(int)

In [27]:
dfUsers.head(5)

,IdUserSteam,IdUser,PurchasedItems,UserUrl
0,76561197960269200,ONIONZZZ,384,http://steamcommunity.com/id/ONIONZZZ
1,76561197960270504,exaqt,209,http://steamcommunity.com/id/exaqt
2,76561197960270742,dvs,36,http://steamcommunity.com/id/dvs
3,76561197960271609,76561197960271609,30,http://steamcommunity.com/profiles/76561197960...
4,76561197960271888,thefourdigit,123,http://steamcommunity.com/id/thefourdigit


**Eliminamos las columnas IdUser, Count y UserUrl del dataframe UrsersItems**

1. *Eliminamos las tres columnas y solo dejamos el **identificador** de usuario.*
2. *Podriamos eliminar la columna NameApp ya que esta información posiblemente la tenemos el el dataframe de Steam Games, sin embargo por ahora la dejamos.*

In [28]:
dfUsersItems = dfUsersItems.drop(columns=["IdUser","Count","UserUrl"])
dfUsersItems.head(3)

,IdUserSteam,IdApp,NameApp,Playtime,PlaytimeLast2W
0,76561197960269200,306660,Ultimate General: Gettysburg,0,0
1,76561197960269200,12150,Max Payne 2: The Fall of Max Payne,0,0
2,76561197960269200,4700,Medieval II: Total War,4,0


In [29]:
dfUsersItems.head(3)

,IdUserSteam,IdApp,NameApp,Playtime,PlaytimeLast2W
0,76561197960269200,306660,Ultimate General: Gettysburg,0,0
1,76561197960269200,12150,Max Payne 2: The Fall of Max Payne,0,0
2,76561197960269200,4700,Medieval II: Total War,4,0


### **Análisis Exploratorio Final** ###
---

*Aquí hacemos una revisión de los datos después de su transformación, con el objetivo de revisar si quedaron datos pendientes por transformar*.

**Nota:** *Posterior a todas las transformaciones se realiza un [EDA](04_EDA.ipynb)
 final.*

*Obtenemos una descripción de las columnas, sus tipos de datos y sus valores no nulos:*

1. *El total de registros es de **5.153.209**, y no se encuentran valores nulos*

In [30]:
dfUsersItems.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5153209 entries, 0 to 5153208
Data columns (total 5 columns):
 #   Column          Dtype 
---  ------          ----- 
 0   IdUserSteam     int64 
 1   IdApp           int64 
 2   NameApp         object
 3   Playtime        int64 
 4   PlaytimeLast2W  int64 
dtypes: int64(4), object(1)
memory usage: 196.6+ MB


*Obtenemos una descripción estadística de los datos numéricos:*

In [31]:
dfUsersItems.describe()

,IdUserSteam,IdApp,Playtime,PlaytimeLast2W
count,5.153209e+06,5.153209e+06,5.153209e+06,5.153209e+06
mean,7.656120e+16,1.784481e+05,9.914951e+02,9.104707e+00
std,3.916645e+07,1.318598e+05,5.418204e+03,1.403926e+02
min,7.656120e+16,1.000000e+01,0.000000e+00,0.000000e+00
25%,7.656120e+16,3.446000e+04,0.000000e+00,0.000000e+00
50%,7.656120e+16,2.144200e+05,3.400000e+01,0.000000e+00
75%,7.656120e+16,2.664300e+05,3.550000e+02,0.000000e+00
max,7.656120e+16,5.307200e+05,6.427730e+05,1.996700e+04


In [32]:
dfUsersItems.columns

Index(['IdUserSteam', 'IdApp', 'NameApp', 'Playtime', 'PlaytimeLast2W'], dtype='object')

### **Exportación de datos a Parquet (Load)** ###
---

*Una vez hemos revisado nuestro dataframe, lo exportamos a un archivo **parquet** para guardar todos los cambios realizados*.

**Nota:** *Los archivos se encuentran en la carpeta **dataout**.*

In [99]:
# dfUsersItems.to_csv("datasets/out_users_items.csv", index=False, encoding="utf-8")

In [100]:
# dfUsers.to_csv("datasets/out_users.csv", index=False, encoding="utf-8")

In [33]:
userItemsTab = pa.Table.from_pandas(dfUsersItems)
dir = "dataout/out_users_items.parquet"
pq.write_table(userItemsTab,dir)

In [34]:
usersTab = pa.Table.from_pandas(dfUsers)
dir2 = "dataout/out_users.parquet"
pq.write_table(usersTab,dir2)